In [1]:
import tensorflow as tf

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten


import cv2
import numpy as np
import matplotlib.pyplot as plt

import PIL
import os
from os import listdir

2024-07-13 02:23:06.746904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 02:23:06.746971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 02:23:06.748525: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-13 02:23:06.753183: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = "Data_Cancer/"
labels = listdir(data_dir)
labels

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [3]:
img_height = 224
img_width = 224
batch_size = 8

In [4]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "training",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

validation_dataset = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "validation",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)



Found 10015 files belonging to 7 classes.
Using 8012 files for training.


2024-07-13 02:23:28.150884: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 02:23:28.175995: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 02:23:28.176108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 02:23:28.178935: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 02:23:28.179010: I external/local_xla/xla/stream_executor

Found 10015 files belonging to 7 classes.
Using 2003 files for validation.


In [5]:
# Normalize the datasets
normalization_layer = layers.Rescaling(1./255)

def normalize_dataset(dataset):
    return dataset.map(lambda x, y: (normalization_layer(x), y))

train_dataset = normalize_dataset(train_dataset)
validation_dataset = normalize_dataset(validation_dataset)

In [6]:
model = Sequential([
    keras.layers.Input((224,224,3)),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Flatten(),
    
    Dense(4096),
    Dense(4096),
    Dense(1000),
    
    Dense(7, activation = "softmax")
    
])

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,00

 Total params: 143,674,247 (548.07 MB)

 Trainable params: 143,674,247 (548.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
with tf.device('/CPU:0'):
    model = Sequential([
        keras.layers.Input((224,224,3)),
        Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
        MaxPooling2D((2,2)),

        Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
        MaxPooling2D((2,2)),

        Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
        MaxPooling2D((2,2)),

        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        MaxPooling2D((2,2)),

        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
        MaxPooling2D((2,2)),

        Flatten(),

        Dense(4096),
        Dense(4096),
        Dense(1000),

        Dense(7, activation = "softmax")
    ])
    
    
    epochs = 10
    model.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy, metrics=["accuracy"])
    history = model.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

Epoch 1/10


I0000 00:00:1720830400.320121   11504 service.cc:145] XLA service 0x7f029c0071c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720830400.320172   11504 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1720830402.477159   11504 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-07-13 02:26:42.481094: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1143476936 exceeds 10% of free system memory.


   1/1002 ━━━━━━━━━━━━━━━━━━━━ 2:36:41 9s/step - accuracy: 0.0000e+00 - loss: 1.9472

2024-07-13 02:26:46.923897: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1143476936 exceeds 10% of free system memory.


   2/1002 ━━━━━━━━━━━━━━━━━━━━ 1:01:08 4s/step - accuracy: 0.1875 - loss: 14.1638   

...

   3/1002 ━━━━━━━━━━━━━━━━━━━━ 1:02:32 4s/step - accuracy: 0.2083 - loss: 15.6453

2024-07-13 02:26:54.433598: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1143476936 exceeds 10% of free system memory.


   4/1002 ━━━━━━━━━━━━━━━━━━━━ 1:03:12 4s/step - accuracy: 0.2031 - loss: 15.3457

2024-07-13 02:26:58.322003: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1143476936 exceeds 10% of free system memory.


  42/1002 ━━━━━━━━━━━━━━━━━━━━ 59:12 4s/step - accuracy: 0.4270 - loss: 5.5648

In [13]:
keras.backend.clear_session()